In [32]:
import joblib
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [51]:
def get_saved_tracks(limit = 50, offset = 0):
    saved_tracks = [ ]
    
    # get initial list of tracks to determine length
    saved_tracks_obj = sp.current_user_saved_tracks(limit = limit, offset = offset)
    num_saved_tracks = saved_tracks_obj['total']
    
    # loop through to get all saved tracked
    while (offset < num_saved_tracks):
        saved_tracks_obj = sp.current_user_saved_tracks(limit = limit, offset = offset)
        
        # add track information to running list
        for track_obj in saved_tracks_obj['items']:
            saved_tracks.append({
                'name': track_obj['track']['name'],
                'artists': ', '.join([artist['name'] for artist in track_obj['track']['artists']]),
                'track_id': track_obj['track']['id']
            })
            
        offset += limit
        
    return saved_tracks

def get_audio_features(track_ids):
    saved_tracks_audiofeat = [ ]
    
    # iterate through track_ids in groups of 50
    for ix in range(0,len(track_ids),50):
        audio_feats = sp.audio_features(track_ids[ix:ix+50])
        saved_tracks_audiofeat += audio_feats
        
    return saved_tracks_audiofeat

def save_cluster_tracks_to_playlist(playlist_name, track_ids):
    # get all of the users playlists
    all_playlists = get_all_user_playlists()
    
    # check if playlist already exists
    if (playlist_name not in [playlist['name'] for playlist in all_playlists]):
        playlist = sp.user_playlist_create(user = user_id, name = playlist_name, public = True)
    else:
        playlist_id = [playlist['id'] for playlist in all_playlists if playlist['name'] == playlist_name][0]
        playlist = sp.user_playlist(user = user_id, playlist_id = playlist_id)

    # remove any existing tracks in playlist
    while (playlist['tracks']['total'] > 0):
        sp.user_playlist_remove_all_occurrences_of_tracks(user_id, playlist['id'], \
                                                          tracks = [track['track']['id'] for track in \
                                                                    playlist['tracks']['items']])
        playlist = sp.user_playlist(user = user_id, playlist_id = playlist_id)

    # add tracks from cluster
    sp.user_playlist_add_tracks(user_id, playlist_id = playlist['id'], tracks = track_ids)
    
def get_all_user_playlists(playlist_limit = 50, playlist_offset = 0):
    # get initial list of users playlists (first n = playlist_limit), determine total number of playlists
    playlists_obj = sp.user_playlists(user_id, limit = playlist_limit, offset = playlist_offset)
    num_playlists = playlists_obj['total']

    # start accumulating playlist names and ids
    all_playlists = [{'name': playlist['name'], 'id': playlist['id']} for playlist in playlists_obj['items']]
    playlist_offset += playlist_limit

    # continue accumulating through all playlists
    while (playlist_offset < num_playlists):
        playlists_obj = sp.user_playlists(user_id, limit = playlist_limit, offset = playlist_offset)
        all_playlists += [{'name': playlist['name'], 'id': playlist['id']} for playlist in playlists_obj['items']]
        playlist_offset += playlist_limit
        
    return(all_playlists)

In [19]:
# Set up the Spotify API client with your credentials
client_id = '988b861a1e9549268a590aeffe09bca7'
client_secret = '24719eab4c9446cd80fa7d7402687bb2'
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
redirect_uri = 'http://localhost:7777/callback'
username = 'drewdifrancesco' 
scope = 'user-library-read'

token = util.prompt_for_user_token(username, scope, client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)

# Create spotipy client
sp = spotipy.Spotify(auth=token)

In [24]:
FEATURE_KEYS = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo','duration_ms','time_signature']

saved_tracks    = get_saved_tracks()
saved_tracks_df = pd.DataFrame(saved_tracks)

# get audio features for saved songs
saved_tracks_audiofeat    = get_audio_features(track_ids = list(saved_tracks_df['track_id']))
saved_tracks_audiofeat_df = pd.DataFrame(saved_tracks_audiofeat).drop(['analysis_url', 'track_href', \
                                                                       'type', 'uri'], axis = 1)

# merge audio features onto tracks df
saved_tracks_plus_df = saved_tracks_df.merge(saved_tracks_audiofeat_df, how = 'left', \
                                             left_on = 'track_id', right_on = 'id').drop('id', axis = 1)

name                  artists                track_id  \
0               mr. sunshine              Arden Jones  2NJefEQeqDKUdyiy38GFoV   
1                  Baby Baby                    Noizu  2fMtXRKJDPtT8Xs4EJkBQh   
2                      Peach           Oscar Scheller  530kyqiGiyVWIArbjrGwKj   
3  Lying Through Their Teeth            Krooked Kings  3hneGCI7tzgFpQeZyv83hq   
4            Carbon Monoxide  Krooked Kings, Day Wave  7lpEGQi4FdLnuT6gRT9sZ1   

   danceability  energy  key  loudness  mode  speechiness  acousticness  \
0         0.589   0.791   11    -3.443     0       0.0521       0.00717   
1         0.627   0.948    7    -4.478     1       0.0428       0.00354   
2         0.751   0.583    7    -4.909     0       0.0387       0.10200   
3         0.442   0.748    0    -7.379     1       0.0399       0.00716   
4         0.529   0.603    2    -6.882     1       0.0284       0.22100   

   instrumentalness  liveness  valence    tempo  duration_ms  time_signature  
0          0.000000    0.1490    0.387  129.978       177231               4  
1          0.551000    0.0599    0.569  125.048       190080               4  
2          0.077000    0.3080    0.688  124.901       212253               4  
3          0.000016    0.1160    0.273  206.629       158544               4  
4          0.019400    0.0882    0.542  166.064       190559               4

In [36]:
scaler = StandardScaler()
norm_d = scaler.fit_transform(saved_tracks_plus_df[FEATURE_KEYS])
norm_d = pd.DataFrame(norm_d, columns = FEATURE_KEYS)
norm_d['name'] = saved_tracks_plus_df['name']
norm_d['artists'] = saved_tracks_plus_df['artists']

In [45]:
loaded_model = joblib.load('model.sav')

In [55]:
norm_d['cluster'] = loaded_model.predict(norm_d[FEATURE_KEYS]) + 1

In [73]:
norm_d[norm_d['cluster']==14]

danceability    energy       key  loudness      mode  speechiness  \
9         0.293900 -1.391636  1.599744 -0.567395  0.763011    -0.700001   
11       -0.807396  0.325501  0.758365 -2.108399  0.763011     0.317462   
90       -0.353529  0.190382  0.197445  0.461540  0.763011    -0.119531   
100      -0.293458  0.184752  1.599744  0.487229  0.763011    -0.593097   
109       0.267202 -0.946869  1.319284 -0.171440  0.763011    -0.643736   
...            ...       ...       ...       ...       ...          ...   
1669     -1.241239 -1.036948  1.038824 -0.279677  0.763011    -0.721569   
1683      2.055972 -0.997539  0.197445 -0.836274  0.763011    -0.290202   
1700     -0.193340 -1.155177  1.319284 -0.014908  0.763011    -0.324899   
1714      0.086990  0.657668  0.758365  1.181177  0.763011    -0.611852   
1722      0.200456  0.590109  0.758365 -0.156712  0.763011    -0.615603   

      acousticness  instrumentalness  liveness   valence     tempo  \
9         0.596309         -0.483034 -0.520236 -1.116523 -1.549768   
11        0.414913          1.226935 -0.616124 -0.897784 -1.050081   
90       -0.775495         -0.483066 -0.744701 -0.254431 -0.842017   
100      -0.764914         -0.268641 -0.273252 -0.914940  0.768431   
109      -0.557820         -0.483070  1.513751  0.234517 -0.816218   
...            ...               ...       ...       ...       ...   
1669      0.917530         -0.438866 -0.617577 -1.103656 -0.660819   
1683     -0.798169         -0.481434 -0.682955 -0.425992 -0.053338   
1700     -0.778896         -0.483070 -0.611039 -1.159414  0.376672   
1714     -0.921670         -0.483054 -1.053432 -0.790558 -0.086928   
1722     -0.766425         -0.482539 -0.091646 -0.263009 -0.617342   

      duration_ms  time_signature                             name  \
9       -0.131999        0.146081  She Don't Wear the Same Clothes   
11      -0.337451        0.146081                       Satellites   
90      -1.072767        0.146081                     Far Rockaway   
100     -1.869859        0.146081                          Formula   
109     -1.262480        0.146081                             F150   
...           ...             ...                              ...   
1669     0.716413        0.146081                      Dory Previn   
1683     0.207649        0.146081                       Down Below   
1700     0.166140        0.146081                 Scott and Ramona   
1714     0.413070        0.146081                      Hollow Life   
1722     0.450773        0.146081          Stay (feat. Maty Noyes)   

               artists  cluster  
9              Molokai       14  
11               faang       14  
90              KAMAUU       14  
100           Labrinth       14  
109          diveliner       14  
...                ...      ...  
1669    Camera Obscura       14  
1683       Roddy Ricch       14  
1700      Lil Uzi Vert       14  
1714      Coast Modern       14  
1722  Kygo, Maty Noyes       14  

[136 rows x 16 columns]